In [6]:
((52-4)/52)**48+58-4

54.02144958527423

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
from scipy.special import jv

In [2]:
from nomaenv import NomaEnv
from algorithms.baselines import RandomScheduler, EarliestDeadlineFirstScheduler, SlottedAloha

In [3]:
import random
import torch

In [4]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [3]:
k = 4
deadlines = np.array([5 for _ in range(k)])
offsets = np.zeros(k)
arrival_probs = np.array([0.8 for _ in range(k)])
period = 5

env = NomaEnv(k,
              deadlines,
              lbda=1,
              period=period,
              arrival_probs=arrival_probs,
              offsets=offsets,
              episode_length=10,
              max_simultaneous_devices=3,
              traffic_model='aperiodic',
              channel_model='collision',
              distances=65,
              path_loss=True,
              shadowing=False,
              fast_fading=True,
              verbose=True
             )

Distances: [65. 65. 65. 65.]
Gain UE: [-90.70970328 -90.70970328 -90.70970328 -90.70970328]
n:1272.0, a:0.9999441788794395


In [4]:
done = False
obs = env.reset()
while not done:
    ac = input()
    ac = np.array([float(a) for a in ac])
    obs, reward, done = env.step(ac)

0000
Timestep 1
State [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Observation [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Action [0. 0. 0. 0.]
Devices polled []
Next state [[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Agent next obs [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Reward 0.0
Received packets [1. 1. 0. 0.]
Number of discarded packets 0

0000
Timestep 2
State [[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Observation [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Action [0. 0. 0. 0.]
Devices polled []
Next state [[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
Agent next obs [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Reward 0.0
Received packets [1. 2. 1. 1.]
Number of discarded packets 0

1111
Timestep 3
State [[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 

KeyboardInterrupt: Interrupted by user

In [6]:
devices_polled = np.nonzero(np.array([0]))[0]

In [7]:
next_obs = np.random.binomial(1, 0.2, (4,5))

In [8]:
next_obs

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0]])

In [12]:
next_obs[devices_polled] = np.ones((4,5))[devices_polled]

In [13]:
next_obs

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0]])

In [20]:
next_obs[~devices_polled] = 0

In [26]:
np.setdiff1d(np.arange(env.k), devices_polled)

array([0, 1, 2, 3])

In [24]:
next_obs[~devices_polled]

array([], shape=(0, 5), dtype=int64)

In [21]:
next_obs

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0]])

In [18]:
next_obs.sum(1).nonzero()[0]

array([0, 3])

In [5]:
obs = env.reset()

In [6]:
pg, h_coeffs = env.compute_channel_components()

In [7]:
env.decode_signal(np.array([0, 1, 1, 1]), pg, h_coeffs)

attempts: [1 2 3], eta: [3.64429670e-06 2.68336215e-06 2.45468411e-06 4.90973028e-06], decoding order: [3 1 2]
fast fading ||h||^2: [21.50720906 15.83615042 14.48658233 28.97530147]
Device: 3, sinr 198.57428443327942, interference: [1 2]
epsilon: 0.0, decoded: 1
Device: 1, sinr 290.9828492859938, interference: [2]
epsilon: 0.0, decoded: 1
Device: 2, sinr 5109.604015368565, interference: []
epsilon: 0.0, decoded: 1


(array([3, 1, 2]), [198.57428443327942, 290.9828492859938, 5109.604015368565])

### Test baselines

In [5]:
k = 8
deadlines = np.array([5 for _ in range(k)])
offsets = np.zeros(k)
arrival_probs = np.array([0.8 for _ in range(k)])
period = 5

env = NomaEnv(k,
              deadlines,
              lbda=1/9.3,
              period=period,
              arrival_probs=arrival_probs,
              offsets=offsets,
              episode_length=200,
              max_simultaneous_devices=3,
              traffic_model='aperiodic',
              channel_model='collision',
              distances=65,
              path_loss=True,
              shadowing=False,
              fast_fading=True,
              verbose=False
             )

In [6]:
random_scheduler = RandomScheduler(env)

In [7]:
random_scheduler.run(500)

KeyboardInterrupt: 

In [11]:
edf_scheduler = EarliestDeadlineFirstScheduler(env, True)

In [12]:
edf_scheduler.run(100)

Number of received packets: 40867.0


1.0

In [4]:
aloha = SlottedAloha(env, 0.5)

In [5]:
transmission_probs_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
cv = aloha.get_best_transmission_probs(50, transmission_probs_list)

In [6]:
cv

[0.36314019784610885,
 0.5743266363591925,
 0.6615205923631612,
 0.6845384087118611,
 0.5797185832120331,
 0.44992878542311276,
 0.2828614152718152,
 0.20614685245577624,
 0.12081692913385822,
 0.0827334630350195]

In [8]:
aloha.transmission_prob = transmission_probs_list[np.argmax(cv)]

In [12]:
aloha.run(500)

0.6706715686274509

In [5]:
from algorithms.dqn import *

In [6]:
k = 4
deadlines = np.array([6 for _ in range(k)])
offsets = np.zeros(k)
arrival_probs = np.array([0.8 for _ in range(k)])
period = 5

env = NomaEnv(k,
              deadlines,
              lbda=1/9.3,
              period=period,
              arrival_probs=arrival_probs,
              offsets=offsets,
              episode_length=200,
              max_simultaneous_devices=3,
              traffic_model='aperiodic',
              channel_model='collision',
              distances=65,
              path_loss=True,
              shadowing=False,
              fast_fading=True,
              verbose=False
             )

In [7]:
dqn = DQN(
    env,
    replay_start_size=500,
    replay_buffer_size=1000000,
    gamma=0.99,
    update_target_frequency=1000,
    minibatch_size=32,
    learning_rate=1e-3,
    update_frequency=1,
    initial_exploration_rate=1,
    final_exploration_rate=0.1,
    adam_epsilon=1e-8,
    loss='huber',
    seed=None,
    )


In [15]:
len(dqn.action_space_combinatorial)

4017

In [16]:
dqn.test(10)

0.816734693877551

In [17]:
dqn.train(10000)

Episode: 0, Train score: 219.00000000000014, Test score: 0.8057764755127669, epsilon:1.0
Episode: 100, Train score: 209.8000000000002, Test score: 0.8090349075975359, epsilon:0.91
Episode: 200, Train score: 209.20000000000016, Test score: 0.7896207584830339, epsilon:0.82
Episode: 300, Train score: 210.90000000000023, Test score: 0.7830150345388054, epsilon:0.73
Episode: 400, Train score: 227.10000000000022, Test score: 0.7378444084278768, epsilon:0.6399999999999999
Episode: 500, Train score: 193.80000000000027, Test score: 0.8090349075975359, epsilon:0.55
Episode: 600, Train score: 218.70000000000024, Test score: 0.8464052287581699, epsilon:0.45999999999999996
Episode: 700, Train score: 195.2000000000002, Test score: 0.8006617038875103, epsilon:0.37
Episode: 800, Train score: 160.70000000000007, Test score: 0.6571428571428571, epsilon:0.2799999999999999


KeyboardInterrupt: 

In [10]:
dqn.test(100)

0.949411621982288

In [19]:
env.deadlines.max()

5

In [9]:
batch = random.sample(list(dqn.replay_buffer._memory.values()), 32)

In [10]:
batch[0]

Transition(state=tensor([-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         1.0000,  1.0000,  1.0000,  0.5000,  1.0000,  1.0000,  0.5000,  1.0000,
         1.0000]), action=tensor([165]), reward=tensor([0.]), state_next=tensor([-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         0.5000,  1.0000,  1.0000,  1.0000,  0.5000,  1.0000,  0.3333,  1.0000,
         0.0000]), done=tensor([0.]))

In [11]:
dqn.replay_buffer.sample(10)

[tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           0.5000,  0.5000,  0.5000,  1.0000,  1.0000,  1.0000,  0.5000,  1.0000,
           1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           1.0000,  1.0000,  0.2500,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           1.0000,  0.5000,  0.5000,  0.5000,  0.2000,  0.5000,  1.0000,  1.0000,
           1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           0.5000,  0.5000,  1.0000,  1.0000,  0.3333,  0.5000,  1.0000,  0.5000,
           2.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           1.0000,  0.3333,  0.3333,  1.0000,  1.0000,  1.0000,  0.5000,  1.0000,
           1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  3.0000, -1.0000

In [5]:
from algorithms.idqn import *

In [6]:
k = 8
deadlines = np.array([6 for _ in range(k)])
offsets = np.zeros(k)
arrival_probs = np.array([0.8 for _ in range(k)])
period = 5

env = NomaEnv(k,
              deadlines,
              lbda=1/9.3,
              period=period,
              arrival_probs=arrival_probs,
              offsets=offsets,
              episode_length=200,
              max_simultaneous_devices=3,
              traffic_model='aperiodic',
              channel_model='collision',
              distances=65,
              path_loss=True,
              shadowing=False,
              fast_fading=True,
              verbose=False
             )

In [7]:
idqn = iDQN(env,
                replay_start_size=500,
                replay_buffer_size=1000000,
                gamma=0.99,
                update_target_frequency=100,
                minibatch_size=64,
                learning_rate=3e-4,
                update_frequency=1,
                initial_exploration_rate=1,
                final_exploration_rate=0.1,
                adam_epsilon=1e-8,
                loss='huber',
                seed=None)

In [8]:
idqn.test_idqn(10)

0.7857142857142857

In [9]:
idqn.train_idqn(10000)

Episode: 0, Train score: 169.10000000000005, Test score: 0.790516206482593
Episode: 100, Train score: 180.20000000000005, Test score: 0.7677578752316244
Episode: 200, Train score: 167.40000000000006, Test score: 0.7844776119402985
Episode: 300, Train score: 144.50000000000003, Test score: 0.7847785402370555
Episode: 400, Train score: 152.0, Test score: 0.7558411214953271
Episode: 500, Train score: 133.8, Test score: 0.7730099502487562
Episode: 600, Train score: 147.50000000000006, Test score: 0.9652226967663209
Episode: 700, Train score: 138.9, Test score: 0.9663518299881936
Episode: 800, Train score: 157.00000000000009, Test score: 0.9787234042553191
Episode: 900, Train score: 183.80000000000015, Test score: 0.9622312608948286
Episode: 1000, Train score: 134.40000000000015, Test score: 0.9846426461901949
Episode: 1100, Train score: 149.10000000000002, Test score: 0.9778621125869703
Episode: 1200, Train score: 132.70000000000022, Test score: 0.9779276517473943
Episode: 1300, Train scor

0.9613027646397707

In [11]:
idqn.test_idqn(100)

0.9719257111124823

In [12]:
idqn.test_idqn(1, verbose=True)

Timestep: 1
Current state: [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]
Action: [0, 1, 1, 1, 1, 0, 0, 0]
Number discarded: 0
Timestep: 2
Current state: [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]]
Action: [0, 1, 1, 1, 1, 0, 0, 0]
Number discarded: 0
Timestep: 3
Current state: [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Action: [0, 1, 1, 1, 1, 0, 0, 1]
Number discarded: 0
Timestep: 4
Current state: [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Action: [0, 0, 0, 0, 1, 0, 0, 1]
Number discarded: 0
Time

Timestep: 178
Current state: [[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]]
Action: [0, 0, 0, 1, 0, 0, 0, 1]
Number discarded: 5
Timestep: 179
Current state: [[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Action: [0, 0, 0, 0, 0, 1, 1, 1]
Number discarded: 5
Timestep: 180
Current state: [[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Action: [0, 0, 1, 0, 1, 1, 0, 1]
Number discarded: 5
Timestep: 181
Current state: [[0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Action: [0, 0, 1, 0, 1, 0, 1, 0]
Number discarded

0.9693251533742331

In [9]:
a = np.random.binomial(1, 0.3, (4,5))

In [13]:
np.concatenate([a, np.ones((4,1))], axis=1)

array([[0., 1., 1., 0., 0., 1.],
       [0., 0., 1., 0., 1., 1.],
       [0., 1., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [15]:
env.reset()

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [16]:
env.last_feedback

0